In [36]:
import json
from transformers import XLNetTokenizer, XLNetForQuestionAnsweringSimple
from torch.nn import functional as F
import torch
import time

In [2]:
x = open("dataset.json")

In [3]:
# import the demo JSON dataset
data = json.load(x)['data']

In [5]:
def get_subjects():
    for i in range(len(data)):
        subject = data[i]['subject_name']
        print(f'{i+1}. {subject}')

In [6]:
def print_topics(data):
    for i in range(len(data)):
        topic = data[i]['topic_name']
        print(f'\n{i+1}. {topic}')

In [43]:
def get_description(top_num, data):
    global continue_param
    global context
    if type(data['description']) == list:
        print_topics(data['description'])
        topic_num = int(input('\n\n Which topic you want help with ? \n\n'))
        get_description(topic_num, data['description'][topic_num - 1])
    else:
        continue_param = False
        context = data['description']
        topic_name = data['topic_name']
        print(f'{topic_name}:')
        print(context)

In [14]:
# Initializing tokenizer and model from fine-tuned model folder
tokenizer = XLNetTokenizer.from_pretrained("fine-tuned-model")
model = XLNetForQuestionAnsweringSimple.from_pretrained("fine-tuned-model", return_dict = True)

In [17]:
# getting the answer using the fine-tuned model
def get_answer(que, text):
    inputs = tokenizer.encode_plus(que, text, return_tensors='pt')
    output = model(**inputs)
    start_max = torch.argmax(F.softmax(output.start_logits, dim = -1))
    end_max = torch.argmax(F.softmax(output.end_logits, dim=-1)) + 1 
    
    answer = tokenizer.decode(inputs["input_ids"][0][start_max : end_max])
    print(answer)

In [48]:
continue_param = True
context = ""

print('The list of subjects available: \n')
get_subjects()
time.sleep(1)

subject_num = int(input('\n\n In which subject can I help you today !? \n\n'))

subject_name = data[subject_num-1]['subject_name']
print(f'The list of topics available in {subject_name}')
print_topics(data[subject_num - 1]['topics'])
time.sleep(1)

topic_num = int(input('\n\n Which topic you want help with ? \n\n'))
time.sleep(1)

while continue_param == True:
    get_description(topic_num, data[subject_num - 1]['topics'][topic_num - 1])

question = str(input('\n\n What is your question ? \n\n'))
get_answer(question, context)

The list of subjects available: 

1. Geography
2. History


 In which subject can I help you today !? 

1
The list of topics available in Geography

1. Introduction

2. Natural Hazards


 Which topic you want help with ? 

2

1. Introduction

2. Types of Natural Hazards


 Which topic you want help with ? 

2

1. Techtonic Hazards

2. Atmospheric Hazards

3. Biological Hazards

4. Geomorphological hazards


 Which topic you want help with ? 

3
Biological Hazards:
Biological health hazards include bacteria, viruses, parasites and moulds or fungi. They can pose a threat to human health when they are inhaled, eaten or come in contact with skin. They can cause illness such as food poisoning, tetanus, respiratory infections or parasite infection.


 What is your question ? 

what can cause food poisoning?

